Clade identifier

Needs to identify the clades that each strain belongs to in accordance with the naming scheme adopted by
Manning 2008. PNAS. Variation in virulence among clades of Escherichia coli O157:H7 associated with disease outbreaks.

This program will identify the SNP sites on the different genomes and then turn them into an array. Then, the program will use those to call what genotype they are.

First - we 

In [1]:
import os
import Bio as bio
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastnCommandline as blastn
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqRecord import SeqRecord
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Style
from openpyxl import load_workbook

#important file locations:
sakai = './Clade_data/sakai.fasta'
cdata = './Clade_data/'
csnps = './Clade_data/SNP_seqs/'
cblast = './Clade_data/SNP_BLAST/'

In [ ]:
#only one genome, can just take it from parser
#build a dictionary of locations: 'gene':'location'(string)

seqlocs = {}

for item in SeqIO.parse(cdata+'sakai.fasta','fasta'):
    sakaigenome = item.seq
for item in SeqIO.parse(cdata+'sakai.gb','gb'):
    for feat in item.features:
        feats = dict(feat.qualifiers)
        location = str(feat.location)
        int1 = int(location.split('[')[1].split(':')[0])
        int2 = int(location.split(':')[1].split(']')[0])
        if location.split('(')[1] == '-)':
            locints = (int2, int1)
        else:
            locints = (int1, int2)
        if ('gene') in feats:
            genename = ''.join(feats['gene'])
            if not genename in seqlocs:
                seqlocs[genename]=locints
                
print(seqlocs)

#checksum - 5498450
print('Checksum - 5498450 bases expected: '+str(len(sakaigenome))+' bases found.\n\n')


In [156]:
#DOES NOT WORK - genome does not align. Only around 50% accuracy. Must refine!
#Will try to use genbank file to extract gene specific data.

#Checksum: extracting releavant locations from sakai genome for blast/muscle
#Reads .xlsx file MSU_2096_20SNPs.xlsx, searches the genome, then spits out the two values.
#First - load genome data as a large string

#only one genome, can just take it from parser
for item in SeqIO.parse(cdata+'sakai.fasta','fasta'):
    sakaigenome = item.seq

#checksum - 5498450
print('Checksum - 5498450 bases expected: '+str(len(sakaigenome))+' bases found.\n\n')

snpxl = load_workbook(filename = cdata+'MSU_2096_20SNPs.xlsx')
snpsheet = snpxl['Sheet1']
print('Format: Locus Expect/Sakai | []/[]')
counter = 0
identities = 0
for i in range(2,98):
    locus = snpsheet['B'+str(i)].value
    expect = snpsheet['E'+str(i)].value
    sakai = sakaigenome[int(snpsheet['D'+str(i)].value)-1]
    counter = counter+1
    if expect == sakai:
        identities = identities+1
    print(locus+' \t'+expect+'/'+sakai+'\t '+str(identities/counter))
print(str(identities)+' of '+str(counter)+' ids.')


Checksum - 5498450 bases expected: 5498450 bases found.


Format: Locus Expect/Sakai | []/[]
ECs0127 	C/C	 1.0
ECs0333 	T/T	 1.0
ECs0472 	G/C	 0.6666666666666666
ECs0495 	C/C	 0.75
ECs0517 	A/A	 0.8
ECs0593 	T/T	 0.8333333333333334
ECs0606 	A/T	 0.7142857142857143
ECs0625 	C/C	 0.75
ECs0654 	T/A	 0.6666666666666666
ECs0654 	T/A	 0.6
ECs0655 	G/C	 0.5454545454545454
ECs0712 	T/T	 0.5833333333333334
ECs0721 	G/C	 0.5384615384615384
ECs0915 	G/C	 0.5
ECs0942 	A/A	 0.5333333333333333
ECs1028 	C/C	 0.5625
ECs1262 	T/A	 0.5294117647058824
ECs1272 	T/T	 0.5555555555555556
ECs1282 	C/C	 0.5789473684210527
ECs1496 	T/T	 0.6
ECs1860 	G/C	 0.5714285714285714
ECs1895 	T/T	 0.5909090909090909
ECs2071 	T/A	 0.5652173913043478
ECs2082 	G/C	 0.5416666666666666
ECs2357 	C/C	 0.56
ECs2375 	C/C	 0.5769230769230769
ECs2521 	T/T	 0.5925925925925926
ECs2598 	C/G	 0.5714285714285714
ECs2696 	C/G	 0.5517241379310345
ECs2775 	G/G	 0.5666666666666667
ECs2852 	T/A	 0.5483870967741935
ECs3022 	T/A	 0.53125
ECs302

In [168]:
#grab sequence by gene, then find the snp location!
#This is a kind of checksum to make sure the right sequences are being pulled at the right index.



snpxl = load_workbook(filename = cdata+'MSU_2096_20SNPs.xlsx')
snpsheet = snpxl['Sheet1']

for item in SeqIO.parse(cdata+'sakai.fasta','fasta'):
    sakaigenome = item.seq
print('Checksum - 5498450 bases expected: '+str(len(sakaigenome))+' bases found.\n\n')





print('Format: Locus Expect/Sakai | []/[]')
counter = 0
identities = 0
for i in range(2,98):
    locus = snpsheet['B'+str(i)].value
    expect = snpsheet['E'+str(i)].value
    geneloc = seqlocs[locus]
    genelen = abs(geneloc[0]-geneloc[1])
    if int(snpsheet['C'+str(i)].value) < genelen:
        if geneloc[0]<geneloc[1]:
            geneseq = sakaigenome[geneloc[0]-1:geneloc[1]+1]
            posneg = 'positive'
        else:
            geneseq = sakaigenome[geneloc[1]-1:geneloc[0]+1].reverse_complement()
            posneg = 'negative'
    else:
        print('NOT COOL! NOT COOL, MAN! Skipping '+locus, str(genelen), snpsheet['C'+str(i)].value)
        continue
    if len(expect) == 1:
        sakai = geneseq[snpsheet['C'+str(i)].value]
    if len(expect) == 2:
        sakai = geneseq[(snpsheet['C'+str(i)].value):(snpsheet['C'+str(i)].value+2)]
    counter = counter+1
    if expect == sakai:
        identities = identities+1
    print(locus+'\t'+expect+'/'+sakai+'\t',geneloc,'\t'+posneg+'\t'+str(identities/counter))
print(str(identities)+' of '+str(counter)+' ids.')

Checksum - 5498450 bases expected: 5498450 bases found.


Format: Locus Expect/Sakai | []/[]
ECs0127	C/C	 (141454, 143005) 	positive	1.0
ECs0333	T/T	 (351026, 351953) 	positive	1.0
ECs0472	G/G	 (503108, 501536) 	negative	1.0
ECs0495	C/C	 (527966, 529838) 	positive	1.0
ECs0517	A/A	 (551825, 552473) 	positive	1.0
ECs0593	T/T	 (651425, 652118) 	positive	1.0
ECs0606	A/A	 (673459, 672997) 	negative	1.0
ECs0625	C/C	 (695812, 699694) 	positive	1.0
ECs0654	T/T	 (730925, 729392) 	negative	1.0
ECs0654	T/T	 (730925, 729392) 	negative	1.0
ECs0655	G/G	 (731859, 730935) 	negative	1.0
ECs0712	T/T	 (788855, 789764) 	positive	1.0
ECs0721	G/G	 (798793, 796594) 	negative	1.0
ECs0915	G/G	 (1003323, 1001997) 	negative	1.0
ECs0942	A/A	 (1026908, 1028390) 	positive	1.0
ECs1028	C/C	 (1133391, 1133961) 	positive	1.0
ECs1262	T/T	 (1323125, 1321781) 	negative	1.0
ECs1272	T/T	 (1337360, 1338893) 	positive	1.0
ECs1282	C/C	 (1349939, 1353746) 	positive	1.0
ECs1496	T/T	 (1536444, 1537689) 	positive	1.0
ECs1860	G/G	 

In [251]:
#Pull the 50 bases in front of and behind the target SNP.
#Some SNPs are too close to end, so must reference whole genome sequence

#51st or 51-52nd position of resulting string, (1st or 1st+2nd position of the downstream fragment made)
#will have the code for the snp. Has 50 flanking bases BOTH up- and down-stream, so 101b long sequences have snp length 1,
#102 have snp length 2, and so on.




snpxl = load_workbook(filename = cdata+'MSU_2096_20SNPs.xlsx')
snpsheet = snpxl['Sheet1']

for item in SeqIO.parse(cdata+'sakai.fasta','fasta'):
    sakaigenome = item.seq
print('Checksum - 5498450 bases expected: '+str(len(sakaigenome))+' bases found.\n\n')

counter = 0
identities = 0
for i in range(2,98):
    locus = snpsheet['B'+str(i)].value
    name = snpsheet['A'+str(i)].value
    expect = snpsheet['E'+str(i)].value
    geneloc = seqlocs[locus]
    genelen = abs(geneloc[0]-geneloc[1])
    if int(snpsheet['C'+str(i)].value) < genelen:
        if geneloc[0]<geneloc[1]:
            snploc = geneloc[0]-1+int(snpsheet['C'+str(i)].value)
            posneg = 'positive'
        else:
            snploc = geneloc[0]-(int(snpsheet['C'+str(i)].value))
            posneg = 'negative'
    else:
        print('NOT COOL! NOT COOL, MAN! Skipping '+locus, str(genelen), snpsheet['C'+str(i)].value)
        continue
    if (len(expect) == 1) and (posneg=='positive'):
        sakaiup = sakaigenome[(snploc-50):(snploc)]
        sakaidown = sakaigenome[(snploc):(snploc+51)]
    #index error in excel spreadsheet requires the use of a small adjustment here. Also note down/up reversed.
    if (len(expect) == 1) and (posneg=='negative'):
        sakaidown = sakaigenome[(snploc-50):(snploc+1)].reverse_complement()
        sakaiup = sakaigenome[(snploc+1):(snploc+51)].reverse_complement()
    if len(expect) == 2:
        sakaidown = sakaigenome[(snploc-50):(snploc+2)].reverse_complement()
        sakaiup = sakaigenome[(snploc+2):(snploc+52)].reverse_complement()
    print(len(sakaiup), len(sakaisnp), len(sakaidown), locus, posneg, expect, sakaidown)
    counter = counter+1
    if sakaidown[0] == expect:
        identities = identities + 1
        print(locus, 'OKAY!')
    if sakaidown[0:2] == expect:
        identities = identities + 1
        print(locus, 'OKAY!')
    queryseq = SeqRecord(Seq(str(sakaiup+sakaidown), generic_dna))
    queryseq.name = name
    queryseq.id = name
    queryseq.description = locus
    queryseqlist = list(queryseq)
    handle = open(csnps+name+'.fasta', 'w')
    SeqIO.write(queryseq, handle, "fasta")
    handle.close
    
    #!!!!!!!!!!!!!!!note - need to run block of code below to get final sequence. No idea why.

Checksum - 5498450 bases expected: 5498450 bases found.


50 1 51 ECs0127 positive C CGAAGGCAATGTCAGTGAAGTGCTGGTGAAGTTTAATCACGACGCACCGAA
ECs0127 OKAY!
50 1 51 ECs0333 positive T TTTCTGAAATGGGTATTACGCTGGCCGCATTGTCCCAGAATTTAAAATCTC
ECs0333 OKAY!
50 1 51 ECs0472 negative G GAGCAATTATTATGAGTTCGTCGACTGACGATATTTCAGTGTCATTGAATT
ECs0472 OKAY!
50 1 51 ECs0495 positive C CCAGATGGGGATGACCGCCGATCAGTACGCCCAGGCGCTGCGTAACCAGCT
ECs0495 OKAY!
50 1 51 ECs0517 positive A AGCGATCCACTCTCAGTATTAAGAGAGATATTAATTCATGTTCTTGAATCC
ECs0517 OKAY!
50 1 51 ECs0593 positive T TATCATTACACCGCCACTATTTGTTAGTGAACCCAAAAGCGAAAATACTTT
ECs0593 OKAY!
50 1 51 ECs0606 negative A ACCCGTTTCACAGGGAACCCTGACACCAGAGCTGTACCAGGAAACGCTGGC
ECs0606 OKAY!
50 1 51 ECs0625 positive C CGGATGAACACGGTGATTTGAGTATTGAGATCCTCGCCAATAAACAGCATT
ECs0625 OKAY!
50 1 51 ECs0654 negative T TCGATCGCAAACTGTGCGCCAACCTGGAAGAAGCGATTCGTCGCTCTGGTT
ECs0654 OKAY!
50 1 51 ECs0654 negative T TCCTGGCGTCCAGCTCCCTGAGTGATTGCCATGCGCCGCTGGTAGAACACA
ECs0654 OKAY!
50 1 51 ECs0

In [252]:
queryseq = SeqRecord(Seq(str(sakaiup+sakaidown), generic_dna))
queryseq.name = name
queryseq.id = name
queryseq.description = locus
queryseqlist = list(queryseq)
handle = open(csnps+name+'.fasta', 'w')
SeqIO.write(queryseq, handle, "fasta")

1

In [4]:
#Need to BLAST all the SNP sequences against the genomes.
#d - Database
#q - Query
#o - output

d = './BLAST_DB/DavisDB.fasta'

for file in os.listdir(csnps):
    if file.endswith('.fasta'):
        q =csnps+file
        o = cblast+os.path.split(file)[1].split('.')[0]+'.xml'
        os.system(str(blastn(query = q, db = d, out = o, outfmt=5)))

In [2]:
#Extract BLAST hits

#hitdict is a dictionary of all blast hits.
#it stores a list of important data for each hit for calculation.
#hits are checked to be one per contig at end.
#hitdict list order:
#query start, query stop, query length, alignment length,
#number of identical positions, number of 'positive' positions
#number of gaps.
#[qstart,qstop,qlen,alen,ids,pos,gaps]

hitdict = {}

#Exclude the salmonella genomes and S54
exclude = ('S66', 'S69', 'S70', 'S54')

#retrieves xml file, parses with element tree, root = top node
for file in os.listdir(cblast):
    if file.endswith('.xml'):
        querysnp = os.path.splitext(file)[0]
        queryf = os.path.join(cblast+file)
        tree = ET.parse(queryf)
        root = tree.getroot()

#Iteration - query information
#hit- hits within genome
#hsp- alignment infor within a hit
        for iteration in root.findall(".//Iteration"):
            qlen = iteration.find('Iteration_query-len').text
        for hit in root.findall(".//Hit"):
            contig = hit.find('Hit_id').text
            if contig.startswith(exclude):
                continue
            for hsp in hit.findall('.//Hit_hsps/Hsp'):
                hseq = hsp.find('Hsp_hseq').text
                qstart = hsp.find('Hsp_query-from').text
                alen = hsp.find('Hsp_align-len').text
                ids = hsp.find('Hsp_identity').text
                qseq = hsp.find('Hsp_qseq').text
                hseq = hsp.find('Hsp_hseq').text
                midl = hsp.find('Hsp_midline').text
                
            key = contig+'|'+querysnp
            if key in list(hitdict.keys()):
                print('overwrite at '+key)
                
            hitdict[key]=[qstart,qlen,alen,ids,qseq,hseq,midl]

In [51]:
#Process the data into a spreadsheet
#Careful to adjust ends of sequence, then ask if the midline indicates a hit, then store nucleotide
#----------------CAUTION----------------CUIDADO----------------------#
#Tests need to be recalibrated for new sets of calls!!!!! They were made here not to solve all problems,
#but only the problems present in this dataset!!!!!

wb = Workbook()
ws = wb.create_sheet(0)

#Create column and row labels

snps = []
genomes = ['0','1',]

for item in hitdict:
    genomes.append(item.split('(')[0])
    snps.append(item.split('|')[1])
genomes = sorted(set(genomes))
snps = sorted(set(snps))
genomes[0] = 'Sakai'
genomes[1] = 'Variant'

for n in range(0, len(snps)):
    ws.cell(row = (n+2), column = 1).value = snps[n]
    
for n in range(0, len(genomes)):
    ws.cell(row = 1, column = (n+2)).value = genomes[n]

#Add in expected sakai and deviant values
snpxl = load_workbook(filename = cdata+'MSU_2096_20SNPs.xlsx')
snpsheet = snpxl['Sheet1']
for i in range(2,98):
    locus = snpsheet['A'+str(i)].value
    expect = snpsheet['E'+str(i)].value
    variant = snpsheet['F'+str(i)].value
    ws.cell(row = snps.index(locus)+2, column=2).value = expect
    ws.cell(row = snps.index(locus)+2, column=3).value = variant

#hitdictkeys is a dict collected by snp of all of the hits
hitdictkeys = {}
hitdictkeylist = list(hitdict.keys())
for item in hitdictkeylist:
    if not item.split('|')[1] in hitdictkeys:
        hitdictkeys[item.split('|')[1]]=[]
    hitdictkeys[item.split('|')[1]].append(item)

for snp in snps:
    if not (len(hitdictkeys[snp])==(len(genomes)-2)):
        print('WARNING: SNP does not have enough sequences!')
    templist = []
    for hit in hitdictkeys[snp]:
        templist.append(hit.split('(')[0])
    templist = sorted(set(templist))
    if genomes[2:]!=templist:
        print('WARNING: Genome list does not match hit list for',snp)
        print('Missing genome(s):',','.join([x for x in genomes[2:] if x not in templist]))
        print('SNP has',len(templist),'genomes present')
        ws.cell(row = snps.index('81_388')+2, column = genomes.index('S53')+2).value = 'No hit'
    
    #REMINDER - hitdict[key]=[qstart,qlen,alen,ids,qseq,hseq,midl]
    keylist = list(hitdictkeys[snp])
    for key in keylist:
        #position - the expected position is 50, adjusted for hits
        position = 50
        
        qstart = int(hitdict[key][0])
        qlen = int(hitdict[key][1])
        alen = int(hitdict[key][2])
        ids = int(hitdict[key][3])
        qseq = hitdict[key][4]
        hseq = hitdict[key][5]
        midl = hitdict[key][6]
        #locus - snp ID, genome - genome hit is in
        locus = key.split('|')[1]
        genome = key.split('(')[0]
        if (qlen==101) and (alen==101) and ('-' not in (hseq or qseq)):
            ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value = hseq[position]
            continue
        if (qlen==101) and (qstart!=1) and ((alen+(qstart-1))==101) and (' ' not in midl) and ('-' not in (hseq or qseq)):
            position = 50-(qstart-1)
            ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value = hseq[position]
            continue
        if (qlen==102) and (alen==102) and ('-' not in (hseq or qseq)):
            ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value = ''.join(hseq[50:52])
            continue
        #Problem with align 26_510 - insertion upstream of SNP (T a position -2 for SNP in two isolates)
        #This solves that. overkill selectors, basically needs to ignore insertion to get SNP.
        if (qlen==101) and ('-' in qseq) and ('-' not in hseq) and (qseq.index('-')<50) and (qseq.count('-')==1) and (locus=='26_510'):
            position = position+1
            ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value = hseq[position]
            continue
        if ('-' in hseq) and ('-' not in qseq) and (qlen==101):
            position = 50-(qstart-1)
            if hseq[position] == '-':
                position = position+1
            ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value = hseq[position]
            continue
        #no blast hit for SNP area - 81_388/S24 had to be investigated by hand. Has 2 partial hits, neither with SNP region
        if (locus==('81_388') and (genome=='S24')):
            ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value = 'No hit'
            continue
        print('Did not find base for:', key)

#Workbook check - see if values match either SNP or Sakai, flag if not
#li - locus index
#gi - genome index

for locus in snps:
    for genome in genomes[2:]:
        sakai = ws.cell(row = snps.index(locus)+2, column = 2).value
        variant = ws.cell(row = snps.index(locus)+2, column = 3).value
        test =  ws.cell(row = snps.index(locus)+2, column = genomes.index(genome)+2).value
        if (test == sakai) or (test == variant):
            continue
        print('Unexpected base for', locus, 'at', genome, 'Ref/var/test = ', sakai, variant, test)
        #83_1484 SNP that this test returns for this dataset is LEGIT! Woo!


##########################################################
        
#Now need to compare the results from this output to a preexistig matrix of hits, then call what clade they are in.
#File SNP_panel_edited.xlsx has a table of SNPs for this purpose.
#It has one row deleted compared to SNP_panel.xlsx that was gotten from
#Mark Mammel at the FDA in Dec. 2015.

#It may be useful to create a crosstable of %IDs

#First, we will build a series of strings.
#Letters are the respective nucleotides
#where an 'N' stands for 'no call' and is ignored in the calculation if either has one.

#Using the workbook made in the cell above.

#Load new workbook SNP_panel_edited.xlsx

snprefwb = load_workbook(filename = cdata+'SNP_panel_edited.xlsx')
snppanel = snprefwb['genotypes']
#snpids has the SNP names in order on the sheet(best order for identification)
#refdict - has strings of nuclotides for each clade reference. IDs are stored as '# clade' with space separator
#snpdict - strings for each test genome
#cladeids - clade names in order (best order for identification)
#snprow - list of snpnames in workbook from cell above - used to find index of each SNP by name
snpids = []
refdict = {}
snpdict = {}
cladeids = []
snprow = []
for i in range(2,60):
    snpids.append(snppanel.cell(row=i, column=3).value)
for i in range(4,40):
    cladeids.append(snppanel.cell(row=1, column=i).value)
for i in range(4,40):
    claderef = snppanel.cell(row=1, column=i).value
    refdict[claderef]=''
    for j in range(2,60):
        refdict[claderef]=refdict[claderef]+snppanel.cell(row=j, column=i).value

for i in range(2,98):
    snprow.append(snpsheet.cell(row = i, column=1).value)
for i in range(2,88):
    genome = ws.cell(row=1, column=i).value
    snpdict[genome]=''
    for item in snpids:
        if ws.cell(row=snprow.index(item)+2, column=i).value[0] == ' ':
            snpdict[genome]=snpdict[genome]+'-'
        else:
            snpdict[genome]=snpdict[genome]+ws.cell(row=snprow.index(item)+2, column=i).value[0]

#Build an array of values of %IDs for each genome SNP set vs references
#Will use a spreadsheet to store these values. First store each snp in ws2
#Store %ID in sheet ws3
#Could build a regular matrix first, but nah.

ws2 = wb.create_sheet(1)
ws3 = wb.create_sheet(2)

for item in snpids:
    ws2.cell(row = snpids.index(item)+2, column = 1).value = item
for item in genomes:
    ws2.cell(row = 1, column = genomes.index(item)+2).value=item
for item in genomes:    
    for jtem in snpids:
        ws2.cell(row = snpids.index(jtem)+2, column = genomes.index(item)+2).value = snpdict[item][snpids.index(jtem)]

for genome in genomes:
    ws3.cell(row = genomes.index(genome)+2, column = 1).value = genome
for clade in cladeids:
    ws3.cell(row = 1, column = cladeids.index(clade)+2,).value = clade
for genome in genomes:
    for clade in cladeids:
        ids = 0
        for i in range(0,58):
            if (snpdict[genome][i]) == (refdict[clade][i]):
                ids += 1
        pctids = 100*ids/58
        ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).value = pctids
fill1= PatternFill(fill_type='solid', start_color='FF9999', end_color='FF9999')
#Mark the highest values
for genome in genomes:
    pcts = []
    for clade in cladeids:
        pcts.append(float(ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).value))
    bestvalue = max(pcts)
    for clade in cladeids:
        if float(ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).value) == bestvalue:
            ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).fill = fill1
        
    
        
        
        
wb.save('SNP_matrix.xlsx')

Missing genome(s): S53
SNP has 83 genomes present
Unexpected base for 81_388 at S53 Ref/var/test =  A G No hit
Unexpected base for 83_1484 at S01 Ref/var/test =  A C G
Unexpected base for 83_1484 at S02 Ref/var/test =  A C G
Unexpected base for 83_1484 at S03 Ref/var/test =  A C G
Unexpected base for 83_1484 at S04 Ref/var/test =  A C G
Unexpected base for 83_1484 at S05 Ref/var/test =  A C G
Unexpected base for 83_1484 at S06 Ref/var/test =  A C G
Unexpected base for 83_1484 at S07 Ref/var/test =  A C G
Unexpected base for 83_1484 at S08 Ref/var/test =  A C G
Unexpected base for 83_1484 at S09 Ref/var/test =  A C G
Unexpected base for 83_1484 at S10 Ref/var/test =  A C G
Unexpected base for 83_1484 at S11 Ref/var/test =  A C G
Unexpected base for 83_1484 at S12 Ref/var/test =  A C G
Unexpected base for 83_1484 at S13 Ref/var/test =  A C G
Unexpected base for 83_1484 at S14 Ref/var/test =  A C G
Unexpected base for 83_1484 at S15 Ref/var/test =  A C G
Unexpected base for 83_1484 at S16

In [50]:
#Now need to compare the results from this output to a preexistig matrix of hits, then call what clade they are in.
#File SNP_panel_edited.xlsx has a table of SNPs for this purpose.
#It has one row deleted compared to SNP_panel.xlsx that was gotten from
#Mark Mammel at the FDA in Dec. 2015.

#It may be useful to create a crosstable of %IDs

#First, we will build a series of strings.
#Letters are the respective nucleotides
#where an 'N' stands for 'no call' and is ignored in the calculation if either has one.

#Using the workbook made in the cell above.

#Load new workbook SNP_panel_edited.xlsx

snprefwb = load_workbook(filename = cdata+'SNP_panel_edited.xlsx')
snppanel = snprefwb['genotypes']
#snpids has the SNP names in order on the sheet(best order for identification)
#refdict - has strings of nuclotides for each clade reference. IDs are stored as '# clade' with space separator
#snpdict - strings for each test genome
#cladeids - clade names in order (best order for identification)
#snprow - list of snpnames in workbook from cell above - used to find index of each SNP by name
snpids = []
refdict = {}
snpdict = {}
cladeids = []
snprow = []
for i in range(2,60):
    snpids.append(snppanel.cell(row=i, column=3).value)
for i in range(4,40):
    cladeids.append(snppanel.cell(row=1, column=i).value)
for i in range(4,40):
    claderef = snppanel.cell(row=1, column=i).value
    refdict[claderef]=''
    for j in range(2,60):
        refdict[claderef]=refdict[claderef]+snppanel.cell(row=j, column=i).value

for i in range(2,98):
    snprow.append(snpsheet.cell(row = i, column=1).value)
for i in range(2,88):
    genome = ws.cell(row=1, column=i).value
    snpdict[genome]=''
    for item in snpids:
        if ws.cell(row=snprow.index(item)+2, column=i).value[0] == ' ':
            snpdict[genome]=snpdict[genome]+'-'
        else:
            snpdict[genome]=snpdict[genome]+ws.cell(row=snprow.index(item)+2, column=i).value[0]

#Build an array of values of %IDs for each genome SNP set vs references
#Will use a spreadsheet to store these values. First store each snp in ws2
#Store %ID in sheet ws3
#Could build a regular matrix first, but nah.

ws2 = wb.create_sheet(1)
ws3 = wb.create_sheet(2)

for item in snpids:
    ws2.cell(row = snpids.index(item)+2, column = 1).value = item
for item in genomes:
    ws2.cell(row = 1, column = genomes.index(item)+2).value=item
for item in genomes:    
    for jtem in snpids:
        ws2.cell(row = snpids.index(jtem)+2, column = genomes.index(item)+2).value = snpdict[item][snpids.index(jtem)]

for genome in genomes:
    ws3.cell(row = genomes.index(genome)+2, column = 1).value = genome
for clade in cladeids:
    ws3.cell(row = 1, column = cladeids.index(clade)+2,).value = clade
for genome in genomes:
    for clade in cladeids:
        ids = 0
        for i in range(0,58):
            if (snpdict[genome][i]) == (refdict[clade][i]):
                ids += 1
        pctids = 100*ids/58
        ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).value = pctids
fill1= PatternFill(fill_type='solid', start_color='FF9999', end_color='FF9999')
#Mark the highest values
for genome in genomes:
    pcts = []
    for clade in cladeids:
        pcts.append(float(ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).value))
    bestvalue = max(pcts)
    for clade in cladeids:
        if float(ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).value) == bestvalue:
            ws3.cell(row = genomes.index(genome)+2, column = cladeids.index(clade)+2).fill = fill1
        
    
        
        
        
wb.save('SNP_matrix.xlsx')

In [ ]:
check = []
ids = (99, 100, 101)
for item in hitdict:
    if '43_3971' in item:
        print (hitdict[item])
        check.append(item)
        if (hitdict[item][3] == ('101' or '100')) and (hitdict[item][4]==('99' or '100' or '101')):
            check.append(item)
print(len(check))

In [34]:
print(snpdict)
print(refdict)
print(genomes)
print(cladeids)

{'S74': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S02': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S10': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S50': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCATTCGGCCCGCCTCTGCGGCCCCT', 'S43': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCATTCGGCCCGCCTCTGCGGCCCCT', 'S22': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S34': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S25': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S23': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S65': 'GCGGGCGCCGGTGGGCCGAGGCCGCCAGGGGACCATTCGGCCCGCCTCTGCGGCCCCT', 'S45': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCATTCGGCCCGCCTCTGCGGCCCCT', 'S36': 'GCGGGCTCCGGTGGATGAGAGTGACCAGGGACATATTCGGCCCGCCTCTGCGGCCCCT', 'S72': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCATTCGGCCCGCCTCTGCGGCCCCT', 'S15': 'GCGGGCTCCGGTGGGCCGAGGCCGCCAGGGGACCCCAAATTCCGCCTCTGCGGCCCCT', 'S56': 'GCGGGCTCCGGTGGGCCGAGGCCGC

In [36]:
for i in range(len(snpdict['S32'])):
    print(snpdict['S32'][i])

G
C
G
G
G
C
T
C
C
G
G
T
G
G
G
C
C
G
A
G
G
C
C
G
C
C
A
G
G
G
G
A
C
C
C
C
A
A
A
T
T
C
C
G
C
C
T
C
T
G
C
G
G
C
C
C
C
T


In [40]:
for i in range(58):
    print (i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
